In [2]:
%pip install imblearn

  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/235.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/235.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/235.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/235.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/235.6 kB 445.2 kB/s eta 0:00:01
   ----------- --------------------------- 71.7/235.6 kB 491.5 kB/s eta 0:00:01
   ------------------ ------------------- 112.6/235.6 kB 598.8 kB/s eta 0:00:01
   ----------------------- -------------- 143.4/235.6 kB 610.6 kB/s eta 0:00:01
   ----------------------- -------------- 143.4/235.6 kB 610.6 kB/s eta 0:00:01
   ----------------------- -------------- 143.4/235.6 kB 610.6 kB/s eta 0:00:01
   ----------------------- -------------- 143.4/235.6 kB 610.6 kB/s eta 0:00:01
   ------------------------ ------------- 153.6/235.6 kB 340.5 kB/s eta 0:00:01
   ---------

In [5]:
%pip install xgboost

   ---------------------------------------- 0.0/99.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.7 MB 131.3 kB/s eta 0:12:40
   ---------------------------------------- 0.0/99.7 MB 131.3 kB/s eta 0:12:40
   ---------------------------------------- 0.0/99.7 MB 131.3 kB/s eta 0:12:40
   ---------------------------------------- 0.0/99.7 MB 131.3 kB/s eta 0:12:40
   ---------------------------------------- 0.1/99.7 MB 228.2 kB/s eta 0:07:17
   ---------------------------------------- 0.1/99.7 MB 228.2 kB/s eta 0:07:17
   ---------------------------------------- 0.1/99.7 MB 226.0 kB/s eta 0:07:21
   ---------------------------------------- 0.1/99.7 MB 226.0 kB/s eta 0:07:21
   ---------------

In [10]:
%pip install nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.


In [1]:
# importing libraries

import pandas as pd
import numpy as np
import plotly.express as px
import pickle
from imblearn.over_sampling import SMOTEN
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report

In [2]:
# loading data
model_df = pd.read_csv(r'E:\my projects\ICM Data\Copper.csv')

In [3]:
# quarying won and lost
# win/lost
query_df = model_df.query("status == 'Won' or status == 'Lost'")
print(query_df['status'].value_counts())

status
Won     59278
Lost     8741
Name: count, dtype: int64


In [4]:
fig = px.bar(x = query_df['status'].value_counts().values)
fig.show()

In [5]:
# the data highly imbalanced

new_df = pd.read_csv(r'E:\my projects\Copper_log_transformed.csv')

In [12]:
query_df = new_df.drop(['application','Unnamed: 0','item_date'],axis=1)
query_df.sample

<bound method NDFrame.sample of        country status item type  thickness     width  product_ref  \
0         28.0    Won         W   0.693147  7.313220   1670798778   
1         30.0    Won        WI  -0.967584  6.858565       628377   
2         32.0    Won         S   0.832909  7.183112   1668701718   
3         28.0    Won         W   1.386294  7.600902       640665   
4         27.0    Won         S  -0.510826  6.917706       628377   
...        ...    ...       ...        ...       ...          ...   
67593     32.0    Won         W  -0.544727  7.106606       611993   
67594     32.0    Won         W  -0.693147  7.130899       611993   
67595     32.0    Won         W  -0.693147  7.130899       611993   
67596     32.0    Won         W  -0.693147  6.903747       611993   
67597     25.0    Won         S  -0.510826  7.160846       611993   

       delivery date  selling_price  quantity_tons  
0               2021       6.749931       3.988984  
1               2021       6.4676

In [14]:
# model building

x = query_df.drop(['status'],axis=1)
y = query_df['status']

In [15]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [16]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [17]:
x_train.shape,x_test.shape
x_train.sample()

,country,item type,thickness,width,product_ref,delivery date,selling_price,quantity_tons
15034,78.0,W,-0.693147,7.130899,611993,2021,7.068172,4.60517


In [18]:
trans1 = ColumnTransformer([
    ('trans1',OneHotEncoder(sparse_output = False,handle_unknown = 'ignore'),[0,1,4,5])
],remainder = 'passthrough')

In [19]:
x_train_trans = trans1.fit_transform(x_train)
x_train_trans.shape

(50698, 52)

In [20]:
x_test_trans = trans1.transform(x_test)
x_test_trans.shape

(16900, 52)

In [21]:
# sampling
# SMOTEENN - Combination

sampling = SMOTEENN()

x_res,y_res = sampling.fit_resample(x_train_trans,y_train)


In [22]:
# SMOTEN - Over sampling

sampling = SMOTEN()

x_res,y_res = sampling.fit_resample(x_train_trans,y_train)

In [23]:
# training the moedel

model = RandomForestClassifier(random_state=42)

model_1 = XGBClassifier()

model_2 = HistGradientBoostingClassifier(random_state=42)

model_3 = ExtraTreesClassifier()

In [24]:
result = model.fit(x_res,y_res)
result = model_1.fit(x_res,y_res)
result = model_2.fit(x_res,y_res)
result = model_3.fit(x_res,y_res)

In [25]:
# predictions

# Random forest regressor
y_pred = model.predict(x_test_trans)
y_pred_1 = model_1.predict(x_test_trans)
y_pred_2 = model_2.predict(x_test_trans)
y_pred_3 = model_3.predict(x_test_trans)

In [26]:
# classification report

# SMOTEENN - Default data
report = classification_report(y_test,y_pred)
print(report)

report = classification_report(y_test,y_pred_1)
print(report)

report = classification_report(y_test,y_pred_2)
print(report)

report = classification_report(y_test,y_pred_3)
print(report)


              precision    recall  f1-score   support

           0       0.73      0.65      0.69      2143
           1       0.95      0.96      0.96     14757

    accuracy                           0.93     16900
   macro avg       0.84      0.81      0.82     16900
weighted avg       0.92      0.93      0.92     16900

              precision    recall  f1-score   support

           0       0.52      0.60      0.56      2143
           1       0.94      0.92      0.93     14757

    accuracy                           0.88     16900
   macro avg       0.73      0.76      0.75     16900
weighted avg       0.89      0.88      0.88     16900

              precision    recall  f1-score   support

           0       0.44      0.56      0.49      2143
           1       0.93      0.90      0.91     14757

    accuracy                           0.85     16900
   macro avg       0.69      0.73      0.70     16900
weighted avg       0.87      0.85      0.86     16900

              preci

In [27]:
# SMOTEENN - Processed Data
report = classification_report(y_test,y_pred)
print(report)

report = classification_report(y_test,y_pred_1)
print(report)

report = classification_report(y_test,y_pred_2)
print(report)

report = classification_report(y_test,y_pred_3)
print(report)

              precision    recall  f1-score   support

           0       0.73      0.65      0.69      2143
           1       0.95      0.96      0.96     14757

    accuracy                           0.93     16900
   macro avg       0.84      0.81      0.82     16900
weighted avg       0.92      0.93      0.92     16900

              precision    recall  f1-score   support

           0       0.52      0.60      0.56      2143
           1       0.94      0.92      0.93     14757

    accuracy                           0.88     16900
   macro avg       0.73      0.76      0.75     16900
weighted avg       0.89      0.88      0.88     16900

              precision    recall  f1-score   support

           0       0.44      0.56      0.49      2143
           1       0.93      0.90      0.91     14757

    accuracy                           0.85     16900
   macro avg       0.69      0.73      0.70     16900
weighted avg       0.87      0.85      0.86     16900

              preci

In [28]:
# SMOTEN - Default data

report = classification_report(y_test,y_pred)
print(report)

report = classification_report(y_test,y_pred_1)
print(report)

report = classification_report(y_test,y_pred_2)
print(report)

report = classification_report(y_test,y_pred_3)
print(report)

              precision    recall  f1-score   support

           0       0.73      0.65      0.69      2143
           1       0.95      0.96      0.96     14757

    accuracy                           0.93     16900
   macro avg       0.84      0.81      0.82     16900
weighted avg       0.92      0.93      0.92     16900

              precision    recall  f1-score   support

           0       0.52      0.60      0.56      2143
           1       0.94      0.92      0.93     14757

    accuracy                           0.88     16900
   macro avg       0.73      0.76      0.75     16900
weighted avg       0.89      0.88      0.88     16900

              precision    recall  f1-score   support

           0       0.44      0.56      0.49      2143
           1       0.93      0.90      0.91     14757

    accuracy                           0.85     16900
   macro avg       0.69      0.73      0.70     16900
weighted avg       0.87      0.85      0.86     16900

              preci

In [29]:
# SMOTEN - Processed Data
report = classification_report(y_test,y_pred)
print(report)

report = classification_report(y_test,y_pred_1)
print(report)

report = classification_report(y_test,y_pred_2)
print(report)

report = classification_report(y_test,y_pred_3)
print(report)

              precision    recall  f1-score   support

           0       0.73      0.65      0.69      2143
           1       0.95      0.96      0.96     14757

    accuracy                           0.93     16900
   macro avg       0.84      0.81      0.82     16900
weighted avg       0.92      0.93      0.92     16900

              precision    recall  f1-score   support

           0       0.52      0.60      0.56      2143
           1       0.94      0.92      0.93     14757

    accuracy                           0.88     16900
   macro avg       0.73      0.76      0.75     16900
weighted avg       0.89      0.88      0.88     16900

              precision    recall  f1-score   support

           0       0.44      0.56      0.49      2143
           1       0.93      0.90      0.91     14757

    accuracy                           0.85     16900
   macro avg       0.69      0.73      0.70     16900
weighted avg       0.87      0.85      0.86     16900

              preci

In [30]:
# SMOTEN - Processed Data - Has an accuracy of 93%

# User Prediction

In [31]:
x_train.sample()

,country,item type,thickness,width,product_ref,delivery date,selling_price,quantity_tons
60764,28.0,W,1.386294,7.501082,640665,2020,6.214608,4.430817


In [32]:
# Real world Prediction

country = 32
item_type = 'W'
product_ref = 1670798778
delivery_year = 2021

thickness = 0.75
log_thickness = np.log(thickness)

width = 1000
log_width = np.log(width)

quantity_tons = 20
log_quantity = np.log(quantity_tons)

selling_price = 20
log_selling_price = np.log(selling_price)

In [33]:
pred_df = pd.DataFrame([[country,item_type,log_thickness,log_width,product_ref,delivery_year,log_selling_price,log_quantity]],
                       columns = ['country','item type','thickness','width','product_ref','delivery date','selling_price','quantity_tons'])

pred_df

,country,item type,thickness,width,product_ref,delivery date,selling_price,quantity_tons
0,32,W,-0.287682,6.907755,1670798778,2021,2.995732,2.995732


In [34]:
pref_df_trans = trans1.transform(pred_df)

In [35]:
# Saving the Encoder and Model
pickle.dump(model,open('Status_Prediction.pkl','wb'))

In [36]:
pickle.dump(trans1,open('encoder.pkl','wb'))